In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import keras
from tensorflow.keras.utils import to_categorical
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/FinalDatasetWithlabels.csv')#,index_col='time_s',parse_dates=['time_s'])
df

,Unnamed: 0,subj_id,loc_id,time_s,x,y,z,magnitude,label
0,0,id86237981,left_wrist,0.00,-0.270,-0.773,-0.160,0.834284,0.0
1,1,id86237981,left_wrist,0.01,-0.266,-0.738,-0.172,0.803109,0.0
2,2,id86237981,left_wrist,0.02,-0.277,-0.707,-0.168,0.777690,0.0
3,3,id86237981,left_wrist,0.03,-0.277,-0.680,-0.160,0.751485,0.0
4,4,id86237981,left_wrist,0.04,-0.289,-0.656,-0.160,0.734477,0.0
...,...,...,...,...,...,...,...,...,...
2590443,2590443,idff99de96,right_ankle,206.71,0.098,1.059,0.090,1.067326,0.0
2590444,2590444,idff99de96,right_ankle,206.72,0.105,1.070,0.074,1.077683,0.0
2590445,2590445,idff99de96,right_ankle,206.73,0.125,1.094,0.090,1.104790,0.0
2590446,2590446,idff99de96,right_ankle,206.74,0.141,1.129,0.090,1.141325,0.0


In [ ]:
# df = df.drop(['Unnamed: 0','subj_id'], 1)
df = df.drop(['Unnamed: 0'], 1)
df = df.drop(['x','y','z'], 1)

In [ ]:
# 
cleanup_nums = {"loc_id":   {"left_wrist":1,"left_hip":2,"left_ankle": 3, "right_ankle": 4}
                }
df = df.replace(cleanup_nums)                                  
df['loc_id'].unique()

array([1, 2, 3, 4])

In [ ]:
df = df.sort_values(by = ['subj_id', 'time_s'], ignore_index=True) 

In [ ]:
df

,subj_id,loc_id,time_s,magnitude,label
0,id00b70b13,1,0.00,1.330820,0.0
1,id00b70b13,2,0.00,1.112560,0.0
2,id00b70b13,3,0.00,2.674321,1.0
3,id00b70b13,4,0.00,1.060073,1.0
4,id00b70b13,1,0.01,1.343959,0.0
...,...,...,...,...,...
2590443,idff99de96,4,206.74,1.141325,0.0
2590444,idff99de96,1,206.75,0.773546,0.0
2590445,idff99de96,2,206.75,1.504684,0.0
2590446,idff99de96,3,206.75,1.368747,0.0


TRAINING AND TEST DATA 


In [ ]:
df.subj_id.unique()[:25]

array(['id00b70b13', 'id079c763c', 'id1165e00c', 'id1c7e64ad',
       'id1f372081', 'id34e056c8', 'id37a54bbf', 'id3e3e50c7',
       'id4ea159a8', 'id5308a7d6', 'id5993bf4a', 'id650857ca',
       'id687ab496', 'id7c20ee7a', 'id82b9735c', 'id86237981',
       'id8af5374b', 'id8e66893c', 'id9603e9c3', 'ida61e8ddf',
       'idabd0c53c', 'idb221f542', 'idbae5a811', 'idc735fc09',
       'idc91a49d0'], dtype=object)

In [ ]:
df_train = df[df.subj_id.isin( df.subj_id.unique()[:25] ) ] 
df_train

,subj_id,loc_id,time_s,magnitude,label
0,id00b70b13,1,0.00,1.330820,0.0
1,id00b70b13,2,0.00,1.112560,0.0
2,id00b70b13,3,0.00,2.674321,1.0
3,id00b70b13,4,0.00,1.060073,1.0
4,id00b70b13,1,0.01,1.343959,0.0
...,...,...,...,...,...
2026247,idc91a49d0,4,212.09,1.032961,0.0
2026248,idc91a49d0,1,212.10,2.053935,1.0
2026249,idc91a49d0,2,212.10,1.107849,0.0
2026250,idc91a49d0,3,212.10,1.140647,1.0


In [ ]:
df_test = df[df.subj_id.isin( df.subj_id.unique()[25:] )]
df_test

,subj_id,loc_id,time_s,magnitude,label
2026252,idd80ac2b4,1,0.00,0.703896,0.0
2026253,idd80ac2b4,2,0.00,1.038725,0.0
2026254,idd80ac2b4,3,0.00,1.058588,0.0
2026255,idd80ac2b4,4,0.00,0.957778,0.0
2026256,idd80ac2b4,1,0.01,0.708258,0.0
...,...,...,...,...,...
2590443,idff99de96,4,206.74,1.141325,0.0
2590444,idff99de96,1,206.75,0.773546,0.0
2590445,idff99de96,2,206.75,1.504684,0.0
2590446,idff99de96,3,206.75,1.368747,0.0


Every subject was timed for 15minutes 

In [ ]:
from scipy import stats
x_list = []
y_list = []
z_list = []
train_labels = []
new_list = []

window_size = 100
step_size = 50

for i in range(0, df_train.shape[0] - window_size, step_size):
    xs = df_train['magnitude'].values[i: i + 100]
    ys = df_train['loc_id'].values[i: i + 100]
    label = stats.mode(df_train['label'][i: i + 100])[0][0]
    a = np.c_[ xs,ys ]
    new_list.append(a)
    x_list.append(xs)
    y_list.append(ys)
    # z_list.append(zs)
    train_labels.append(label)

In [ ]:
Xtrain = np.array(new_list)

In [ ]:
train_set = pd.concat([train_magnitude,train_locid],1)
train_set

,column,column1
0,"1.330820047940367,1.1125596613215851,2.6743210...","1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,..."
1,"1.2116150378730035,1.2056077305657922,1.695061...","3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,..."
2,"1.6584046550827092,1.3516116306099175,1.030478...","1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,..."
3,"1.0613185195783592,1.057034058107874,1.0401124...","3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,..."
4,"1.7583696994659568,1.054318737384478,1.4067043...","1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,..."
...,...,...
40519,"1.1597180691875073,1.2317796069102622,1.213707...","3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,..."
40520,"1.0772557727856462,1.0232682932642836,3.342228...","1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,..."
40521,"1.1080297829932189,2.2061697577475763,0.734360...","3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,..."
40522,"0.9686051827241066,0.5480127735737553,1.371089...","1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,4,1,2,3,..."


In [ ]:
train_labels

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0

In [ ]:
model_cnn = Sequential() 
model_cnn.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(100,2)))
model_cnn.add(MaxPooling1D(pool_size=1))
model_cnn.add(Flatten())
model_cnn.add(Dense(50, activation='sigmoid'))
model_cnn.add(Dense(1,activation='sigmoid'))

model_cnn.compile(loss='binary_crossentropy', optimizer="adam", metrics = ["accuracy", tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
model_cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 99, 64)            320       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 6336)              0         
                                                                 
 dense_4 (Dense)             (None, 50)                316850    
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 317,221
Trainable params: 317,221
Non-trainable params: 0
________________________________________________

In [ ]:
train_x = np.asarray(Xtrain)
train_y = np.asarray(train_labels)


In [ ]:
model_cnn.fit(train_x, train_y, epochs=1000, batch_size=1000, verbose=2)

Epoch 1/1000
41/41 - 1s - loss: 0.6028 - accuracy: 0.6746 - recall_1: 0.5339 - precision_1: 0.6755 - 686ms/epoch - 17ms/step
Epoch 2/1000
41/41 - 1s - loss: 0.6021 - accuracy: 0.6745 - recall_1: 0.5399 - precision_1: 0.6729 - 671ms/epoch - 16ms/step
Epoch 3/1000
41/41 - 1s - loss: 0.6010 - accuracy: 0.6754 - recall_1: 0.5350 - precision_1: 0.6766 - 676ms/epoch - 16ms/step
Epoch 4/1000
41/41 - 1s - loss: 0.5987 - accuracy: 0.6771 - recall_1: 0.5393 - precision_1: 0.6780 - 681ms/epoch - 17ms/step
Epoch 5/1000
41/41 - 1s - loss: 0.5968 - accuracy: 0.6779 - recall_1: 0.5443 - precision_1: 0.6772 - 671ms/epoch - 16ms/step
Epoch 6/1000
41/41 - 1s - loss: 0.5964 - accuracy: 0.6785 - recall_1: 0.5458 - precision_1: 0.6778 - 685ms/epoch - 17ms/step
Epoch 7/1000
41/41 - 1s - loss: 0.5941 - accuracy: 0.6806 - recall_1: 0.5388 - precision_1: 0.6849 - 679ms/epoch - 17ms/step
Epoch 8/1000
41/41 - 1s - loss: 0.5935 - accuracy: 0.6820 - recall_1: 0.5560 - precision_1: 0.6797 - 682ms/epoch - 17ms/step


In [ ]:
model_cnn.save('keras.h5') 

In [ ]:
from scipy import stats
x_list = []
y_list = []
z_list = []
test_y = []
test_x = []

window_size = 100
step_size = 50

for i in range(0, df_test.shape[0] - window_size, step_size):
    xs = df_test['magnitude'].values[i: i + 100]
    ys = df_test['loc_id'].values[i: i + 100]
    label = stats.mode(df_test['label'][i: i + 100])[0][0]
    a = np.c_[ xs,ys ]
    test_x.append(a)
    test_y.append(label)

In [ ]:
len(test_x)

11282

In [ ]:
test_x = np.asarray(test_x)
test_y = np.asarray(test_y)

scores = model_cnn.evaluate(test_x,test_y, verbose=0)

In [ ]:
scores

[3.687227249145508, 0.514625072479248, 0.4571259617805481, 0.4600277841091156]

In [ ]:
print("%s: %.2f%%" % (model_cnn.metrics_names[1], scores[1]*100))

accuracy: 51.46%


In [ ]:
test_x

array([[[0.70389559, 1.        ],
        [1.0387247 , 2.        ],
        [1.05858821, 3.        ],
        ...,
        [1.47841807, 2.        ],
        [1.33792227, 3.        ],
        [1.32596418, 4.        ]],

       [[1.10468321, 3.        ],
        [1.06194774, 4.        ],
        [0.91278968, 1.        ],
        ...,
        [2.02555918, 4.        ],
        [1.19060699, 1.        ],
        [1.20620272, 2.        ]],

       [[1.18833707, 1.        ],
        [1.48448947, 2.        ],
        [1.33665029, 3.        ],
        ...,
        [0.65879359, 2.        ],
        [0.97458145, 3.        ],
        [0.69992357, 4.        ]],

       ...,

       [[2.55678333, 3.        ],
        [2.63803146, 4.        ],
        [1.53778217, 1.        ],
        ...,
        [1.09960493, 4.        ],
        [0.83600239, 1.        ],
        [1.04295062, 2.        ]],

       [[1.11905317, 1.        ],
        [1.70700059, 2.        ],
        [2.96618695, 3.        ],
        .

In [ ]:
pred = model_cnn.predict(test_x) 
pred = np.argmax(pred, axis = 1)
label = test_y

x = pd.DataFrame(pred,columns= ["preds"])


,preds
0,0
1,0
2,0
3,0
4,0
...,...
11277,0
11278,0
11279,0
11280,0


In [ ]:
test_y

array([0., 0., 0., ..., 1., 1., 1.])